In [1]:
from typing import Callable
from itertools import product

In [2]:
from cpa_utils import *

# Elisabeth-b4 - Filter block

## Attacking the filter S-boxes

In [3]:
traces_path = "..\\acquisition\\carto_eB4-Rnd-3-WhiteningAndFullFilter-1_key_256000_samples_full_filter\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Alternating-Same-And-Varying-Seeds.mat"
key_path = "..\\acquisition\\carto_eB4-Rnd-3-WhiteningAndFullFilter-1_key_256000_samples_full_filter\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Alternating-Same-And-Varying-Seeds.log"

In [4]:
seeds, traces, real_keys = load_data(traces_path, key_path)

In [5]:
with open("correlation_locations_b4_two_last_rounds_hw_x_i.pic", "rb") as r:
    hw_x_i_locations = pic.load(r)

In [6]:
def multiple_locations_and_hypotheses(funs_hypotheses: List[Callable], total_seeds: np.ndarray) -> np.ndarray:
    hypotheses = np.zeros((KEY_WIDTH_B4, len(funs_hypotheses), 16, total_seeds.shape[0]), dtype=int)

    for i, iv in enumerate(total_seeds):
        indices, whitening = chacha_random_b4(iv)
        for keyround_target_idx in range(KEYROUND_WIDTH_B4):
            round_idx = keyround_target_idx // BLOCK_WIDTH_B4
            block_idx = keyround_target_idx % BLOCK_WIDTH_B4
    
            for l, fun_hypo in enumerate(funs_hypotheses):
                for k in range(16):
                    hyp = fun_hypo(round_idx, block_idx, k, whitening)
                    if hyp is not None:
                        hypotheses[indices[keyround_target_idx], l, k, i] = hyp

    return hypotheses

In [7]:
def hyp_even_sbox_output(round_idx: int, block_idx: int, k: int, whitening: List[int]) -> int:
    if block_idx == 6 or block_idx % 2 != 0:
        return None
    return HW[s_boxes_b4[block_idx][(k + whitening[round_idx * BLOCK_WIDTH_B4 + block_idx]) % 16]]

In [8]:
def hyp_hw_x_i(round_idx: int, block_idx: int, k: int, whitening: List[int]) -> int:
    return HW[(k + whitening[round_idx * BLOCK_WIDTH_B4 + block_idx]) % 16]

In [9]:
location_all = np.tile(np.arange(traces[0].shape[1]), (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, 1))

In [11]:
hypotheses_funs = [hyp_even_sbox_output, hyp_hw_x_i]

reconstructed_keys = np.zeros_like(real_keys)
for i in range(reconstructed_keys.shape[0]):
    print(f"Key {i}: ")
    hypotheses = multiple_locations_and_hypotheses(hypotheses_funs, seeds[i])
    #corrs = np.array([[corr_coef_vectorized(hypotheses[l], selected_traces) for l in range(hypotheses.shape[0])] for selected_traces, hypotheses in zip(list_selected_traces, list_hypotheses)])
    corrs = corr_coef_vectorized(hypotheses, traces[i])

    # fig, axs = plt.subplots(corrs.shape[0], 2, figsize=(16, 20))
    # if corrs.shape[0] == 1:
    #     axs = axs[np.newaxis, :]
    # for k in range(corrs.shape[0]):
    #     for l in range(corrs.shape[1]):
    #         axs[k, 0].plot(corrs[k][l], label=str(l))

    #     axs[k, 0].set_ylim([-0.5, 0.5])
    #     axs[k, 0].set_title(hypotheses_names[k])
        
    max_corrs = np.max(corrs, axis=2)
    best_k_per_hyp = np.argmax(max_corrs, axis=1)
    if True:
        mean_corrs = np.mean(max_corrs, axis=0)
        best_k = np.argmax(mean_corrs) # Correlation mean
    else:
        best_k = Counter(best_k_per_hyp).most_common(1)[0][0] # Majority voting
    reconstructed_keys[i][j] = best_k

    # for k in range(max_corrs.shape[0]):
    #     axs[k, 1].plot(max_corrs[k])
    #     axs[k, 1].set_xlabel("Key")
    #     axs[k, 1].set_ylabel("Max correlation")
    #     axs[k, 1].set_title(f"Real key nibble: {real_keys[i][j]}, chosen {best_k_per_hyp[k]}")
    #     axs[k, 1].set_ylim([0, 0.5])
    # plt.show()
    print(f"Chosen {best_k}")

    print()
    print(f"    vs {"".join([hex(k)[2:].upper() for k in real_keys[i]])}")
    print(f"{len(real_keys[i][real_keys[i] != reconstructed_keys[i]])} mistakes on {len(real_keys[i])} nibbles.")
    print()

Key 0: 


MemoryError: Unable to allocate 2.98 PiB for an array with shape (512, 2, 16, 255998, 100000) and data type float64